# Granger Causality

Brief 1-2 sentence description of notebook.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import glob
from collections import defaultdict
import re

In [3]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from scipy import stats
import itertools
from scipy.stats import linregress

In [4]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [5]:
from statsmodels.tsa.stattools import grangercausalitytests


## Inputs & Data

Explanation of each input and where it comes from.

In [6]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

In [7]:
BBOX_TO_ANCHOR=(1.5, 0.9)
LOC='upper right'

In [8]:
ALL_BANDS = ["theta", "beta", "gamma"]
BAND_TO_FREQ = {"theta": {"low_freq": 4, "high_freq": 12}, "beta": {"low_freq": 13, "high_freq": 30}, "gamma": {"low_freq": 30, "high_freq": 70}}

In [9]:
# variables for LFP extraction
FREQ_MIN=0.5
FREQ_MAX=300
NOTCH_FREQ=60
ORIGINAL_SAMPLE_RATE = 20000
RESAMPLE_RATE=1000
TRIAL_DURATION=10

In [10]:
INPUT_VARIABLE = 1

TIME_HALFBANDWIDTH_PRODUCT = 2
TIME_WINDOW_DURATION = 3
TIME_WINDOW_STEP = 1.5 

TRIAL_TIME_STAMP_DURATION = 1000*10

In [11]:
BIN_TO_COLOR = {0: {"baseline": "lightblue", "trial": "blue"}, 1: {"baseline": "lightgreen", "trial": "green"}, 2: {"baseline": "lightcoral", "trial": "red"}}
TRIAL_OR_BASELINE_TO_STYLE = {'baseline': "--", "trial": "-"}
BIN_TO_VELOCITY = {0: "0 to 2.5cm/s", 1: "2.5 to 5cm/s", 2: "5cm/s+"}

In [12]:
NUM_LINES = 3

In [13]:
# Generate colors from the "Blues" colormap
LOSING_COLORS = cm.Oranges(np.linspace(0.5, 1, NUM_LINES))
# Generate colors from the "Blues" colormap
WINNING_COLORS = cm.Blues(np.linspace(0.5, 1, NUM_LINES))
# Generate colors from the "Blues" colormap
REWARDED_COLORS = cm.Greens(np.linspace(0.5, 1, NUM_LINES))
# Generate colors from the "Blues" colormap
OMISSION_COLORS = cm.Reds(np.linspace(0.5, 1, NUM_LINES))

In [14]:
BASELINE_OUTCOME_TO_COLOR = {'lose': "orange",
 'lose_baseline': LOSING_COLORS[0],
 'omission': "red",
 'omission_baseline': "hotpink",
 'rewarded': "green",
 'rewarded_baseline': REWARDED_COLORS[0],
 'win': "blue",
 'win_baseline': WINNING_COLORS[0]}

In [15]:
BASELINE_OUTCOME_TO_COLOR = {'lose_trial': "orange",
 'lose_baseline': LOSING_COLORS[0],
 'omission_trial': "red",
 'omission_baseline': "hotpink",
 'rewarded_trial': "green",
 'rewarded_baseline': REWARDED_COLORS[0],
 'win_trial': "blue",
 'win_baseline': WINNING_COLORS[0]}

In [16]:
COMPETITIVE_OUTCOME_TO_COLOR = {'lose_comp': "orange", 
'lose_non_comp': "yellow",
'omission': "red",
'rewarded': "green",
'win_comp': "blue", 
'win_non_comp': WINNING_COLORS[0]}

In [17]:
TRIAL_OR_BASELINE_TO_STYLE = {'baseline': "--", "trial": "-"}

In [18]:
CHANNEL_MAPPING_DF = pd.read_excel("../../channel_mapping.xlsx")
CHANNEL_MAPPING_DF["Subject"] = CHANNEL_MAPPING_DF["Subject"].astype(str)

TONE_TIMESTAMP_DF = pd.read_excel("../../rce_tone_timestamp.xlsx", index_col=0)
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs


In [19]:
CHANNEL_MAPPING_DF = pd.read_excel("../../channel_mapping.xlsx")
CHANNEL_MAPPING_DF["Subject"] = CHANNEL_MAPPING_DF["Subject"].astype(str)

TONE_TIMESTAMP_DF = pd.read_excel("../../rce_tone_timestamp.xlsx", index_col=0)
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs


In [20]:
FULL_LFP_TRACES = pd.read_pickle("./proc/full_baseline_and_trial_lfp_traces.pkl")

In [21]:
FULL_LFP_TRACES = FULL_LFP_TRACES.drop_duplicates(subset=["recording_file", "time"], keep="first")

In [22]:
GROUPINGS = "competition_closeness"


## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions

- Ideally functions are defined here first and then data is processed using the functions
    - function names are short and in snake case all lowercase
    - a function name should be unique but does not have to describe the function
    - doc strings describe functions not function names

In [23]:
def generate_pairs(lst):
    pairs = []
    n = len(lst)
    for i in range(n):
        for j in range(i+1, n):
            pairs.append((lst[i], lst[j]))
    return pairs

In [24]:
def nested_dict():
    return defaultdict(dict)

triple_nested_dict = defaultdict(nested_dict)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [25]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

## Granger Causality

In [26]:
baseline_cols = FULL_LFP_TRACES.columns

In [27]:
brain_region_cols = [col for col in FULL_LFP_TRACES.columns if "spike_interface" in col]

In [28]:
brain_region_cols

['spike_interface_mPFC',
 'spike_interface_vHPC',
 'spike_interface_BLA',
 'spike_interface_LH',
 'spike_interface_MD']

In [29]:
baseline_pairs = generate_pairs(sorted([col for col in FULL_LFP_TRACES.columns if "baseline_lfp_trace" in col]))
trial_pairs = generate_pairs(sorted([col for col in FULL_LFP_TRACES.columns if "trial_lfp_trace" in col]))

In [30]:
baseline_pairs

[('BLA_baseline_lfp_trace', 'LH_baseline_lfp_trace'),
 ('BLA_baseline_lfp_trace', 'MD_baseline_lfp_trace'),
 ('BLA_baseline_lfp_trace', 'mPFC_baseline_lfp_trace'),
 ('BLA_baseline_lfp_trace', 'vHPC_baseline_lfp_trace'),
 ('LH_baseline_lfp_trace', 'MD_baseline_lfp_trace'),
 ('LH_baseline_lfp_trace', 'mPFC_baseline_lfp_trace'),
 ('LH_baseline_lfp_trace', 'vHPC_baseline_lfp_trace'),
 ('MD_baseline_lfp_trace', 'mPFC_baseline_lfp_trace'),
 ('MD_baseline_lfp_trace', 'vHPC_baseline_lfp_trace'),
 ('mPFC_baseline_lfp_trace', 'vHPC_baseline_lfp_trace')]

In [31]:
def get_granger(row, region_1, region_2, max_lag=3):
    """
    """
    try:
        return grangercausalitytests(np.hstack([row[region_1][np.newaxis].T, row[region_2][np.newaxis].T]), maxlag=[max_lag])[max_lag][0]["ssr_ftest"][0]
    except:
        return np.nan

In [32]:
MAX_LAG = 10

In [ ]:
baseline_or_trial = "trial"
for region_1, region_2 in trial_pairs:
    region_1_name = region_1.split("_")[0]
    region_2_name = region_2.split("_")[0]
    
    
    FULL_LFP_TRACES["{}_{}_{}_granger".format(region_1_name, region_2_name, baseline_or_trial)] = FULL_LFP_TRACES.apply(lambda row: get_granger(row, region_1, region_2, max_lag=MAX_LAG), axis=1)

    FULL_LFP_TRACES["{}_{}_{}_granger".format(region_2_name, region_1_name, baseline_or_trial)] = FULL_LFP_TRACES.apply(lambda row: get_granger(row, region_2, region_1, max_lag=MAX_LAG), axis=1)


Granger Causality
number of lags (no zero) 10
ssr based F test:         F=27.2717 , p=0.0000  , df_denom=9969, df_num=10
ssr based chi2 test:   chi2=273.2910, p=0.0000  , df=10
likelihood ratio test: chi2=269.6197, p=0.0000  , df=10
parameter F test:         F=27.2717 , p=0.0000  , df_denom=9969, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=23.2615 , p=0.0000  , df_denom=9969, df_num=10
ssr based chi2 test:   chi2=233.1050, p=0.0000  , df=10
likelihood ratio test: chi2=230.4269, p=0.0000  , df=10
parameter F test:         F=23.2615 , p=0.0000  , df_denom=9969, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=20.0520 , p=0.0000  , df_denom=9969, df_num=10
ssr based chi2 test:   chi2=200.9424, p=0.0000  , df=10
likelihood ratio test: chi2=198.9482, p=0.0000  , df=10
parameter F test:         F=20.0520 , p=0.0000  , df_denom=9969, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         

In [ ]:
baseline_or_trial = "baseline"
for region_1, region_2 in baseline_pairs:
    region_1_name = region_1.split("_")[0]
    region_2_name = region_2.split("_")[0]
    
    
    FULL_LFP_TRACES["{}_{}_{}_granger".format(region_1_name, region_2_name, baseline_or_trial)] = FULL_LFP_TRACES.apply(lambda row: get_granger(row, region_1, region_2, max_lag=MAX_LAG), axis=1)

    FULL_LFP_TRACES["{}_{}_{}_granger".format(region_2_name, region_1_name, baseline_or_trial)] = FULL_LFP_TRACES.apply(lambda row: get_granger(row, region_2, region_1, max_lag=MAX_LAG), axis=1)

In [ ]:
FULL_LFP_TRACES

In [ ]:
raise ValueError()

In [ ]:
baseline_or_trial = "baseline"
for region_1, region_2 in baseline_pairs:
    region_1_name = region_1.split("_")[0]
    region_2_name = region_2.split("_")[0]
    
    
    FULL_LFP_TRACES["{}_{}_granger".format(region_1_name, region_2_name)] = FULL_LFP_TRACES.apply(lambda row: get_granger(row, region_1, region_2), axis=1)

    FULL_LFP_TRACES["{}_{}_granger".format(region_2_name, region_1_name)] = FULL_LFP_TRACES.apply(lambda row: get_granger(row, region_2, region_1), axis=1)

    

In [ ]:
FULL_LFP_TRACES["BLA_LH_granger"].iloc[5][3][0]

In [ ]:
FULL_LFP_TRACES["LH_BLA_granger"].iloc[0][3][0]

In [ ]:
baseline_or_trial = "baseline"
for region_1, region_2 in baseline_pairs:
    region_1_name = region_1.split("_")[0]
    region_2_name = region_2.split("_")[0]

    FULL_LFP_TRACES["{}_{}_granger".format(region_1_name, region_2_name)] = FULL_LFP_TRACES.apply(lambda row: np.hstack([row[region_1][np.newaxis].T, row[region_2][np.newaxis].T]), axis=1)


    
    break
    FULL_LFP_TRACES["{}_{}_granger".format(region_1_name, region_2_name)] = FULL_LFP_TRACES.apply(lambda row: grangercausalitytests(np.hstack([row[region_1][np.newaxis].T, row[region_2][np.newaxis].T]), maxlag=[3]), axis=1)

    FULL_LFP_TRACES["{}_{}_granger".format(region_2_name, region_1_name)] = FULL_LFP_TRACES.apply(lambda row: grangercausalitytests(np.hstack([row[region_2][np.newaxis].T, row[region_1][np.newaxis].T]), maxlag=[3]), axis=1)
    break

In [ ]:
FULL_LFP_TRACES["BLA_LH_granger"].iloc[0].shape

In [ ]:
example_arr.shape

In [ ]:
example_arr = np.hstack([FULL_LFP_TRACES["BLA_trial_lfp_trace"].iloc[0][np.newaxis].T, FULL_LFP_TRACES["MD_trial_lfp_trace"].iloc[0][np.newaxis].T])

In [ ]:
example_arr = np.hstack([FULL_LFP_TRACES["MD_trial_lfp_trace"].iloc[0][np.newaxis].T, FULL_LFP_TRACES["BLA_trial_lfp_trace"].iloc[0][np.newaxis].T])

In [ ]:
grangercausalitytests(example_arr, maxlag=[3])

In [ ]:
grangercausalitytests(example_arr, maxlag=[3])

In [ ]:
region_1_name

In [ ]:
#perform Granger-Causality test
grangercausalitytests(FULL_LFP_TRACES[[baseline_pairs[0][0], baseline_pairs[0][1]]], maxlag=[3])

In [ ]:
raise ValueError()

In [ ]:
trace_columns = [col for col in channel_map_and_all_trials_df.columns if "trace" in col]

In [ ]:
trace_columns

## Power correlation between brain regions calculation

- Combining the trial/baseline and outcome label for coloring

In [ ]:
channel_map_and_all_trials_df["outcome_and_trial_or_baseline"] = channel_map_and_all_trials_df.apply(lambda x: "_".join([x["trial_outcome"], x["trial_or_baseline"]]), axis=1)

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests



In [ ]:
channel_map_and_all_trials_df.columns

In [ ]:
trace_columns = [col for col in channel_map_and_all_trials_df.columns if "trace" in col]

In [ ]:
trace_columns

In [ ]:
brain_region_pairs = generate_pairs(trace_columns)

In [ ]:
brain_region_pairs

In [ ]:
grangercausalitytests(df[['column1', 'column2']], maxlag=[3])

In [ ]:
channel_map_and_all_trials_df[region_1].iloc[0]

In [ ]:
def granger

In [ ]:
for region_1, region_2 in brain_region_pairs:
    pair_base_name = "{}_{}".format(region_1.strip("trace").strip("_"), region_2.strip("trace").strip("_"))
    print(pair_base_name)
    try:
        
        # granger_value = grangercausalitytests(channel_map_and_all_trials_df[[region_1, region_2]], maxlag=[3])
        channel_map_and_all_trials_df["{}_granger".format(pair_base_name)] = channel_map_and_all_trials_df.apply(lambda row: grangercausalitytests(np.array([row[region_1], row[region_2]]).T, maxlag=[3]), axis=1)
        print()
    except Exception as e: 
        print(e)
    break

In [ ]:
channel_map_and_all_trials_df[""]

In [ ]:
channel_map_and_all_trials_df

In [ ]:
granger_value

- Filtering out the outliers

In [ ]:
band_to_power_correlation = defaultdict(dict)
for band in ALL_BANDS:
    # Getting all the pairs of brain regions
    band_averaged_columns = [col for col in channel_map_and_all_trials_df.columns if "averaged_{}".format(band) in col]
    band_to_power_correlation[band]["brain_region_pairs"] = generate_pairs(band_averaged_columns)
    print(band_to_power_correlation[band]["brain_region_pairs"])

    # Removing rows that are outliers
    filtered_df = channel_map_and_all_trials_df.copy()
    
    for col in band_averaged_columns:
        # filtered_df = filtered_df[filtered_df[col] <= 3]
        # Assuming data is a 1D numpy array
        Q1 = np.percentile(filtered_df[col], 25)
        Q3 = np.percentile(filtered_df[col], 75)
        IQR = Q3 - Q1
        band_to_power_correlation[band]["outlier_removed_df"] = filtered_df[(filtered_df[col] >= Q1 - 1.5 * IQR) & (filtered_df[col] <= Q3 + 1.5 * IQR)]


    
    # Getting the mean and standard deviation
    

In [ ]:
band_to_power_correlation = defaultdict(dict)
for band in ALL_BANDS:
    # Getting all the pairs of brain regions
    band_averaged_columns = [col for col in channel_map_and_all_trials_df.columns if "averaged_{}".format(band) in col]
    band_to_power_correlation[band]["brain_region_pairs"] = generate_pairs(band_averaged_columns)
    print(band_to_power_correlation[band]["brain_region_pairs"])

    # Removing rows that are outliers
    filtered_df = channel_map_and_all_trials_df.copy()
    
    for col in band_averaged_columns:
        # filtered_df = filtered_df[filtered_df[col] <= 3]
        # Assuming data is a 1D numpy array
        Q1 = np.percentile(filtered_df[col], 25)
        Q3 = np.percentile(filtered_df[col], 75)
        IQR = Q3 - Q1
        filtered_df = filtered_df[(filtered_df[col] >= Q1 - 1.5 * IQR) & (filtered_df[col] <= Q3 + 1.5 * IQR)]
    band_to_power_correlation[band]["outlier_removed_df"] = filtered_df

In [ ]:
channel_map_and_all_trials_df.shape

In [ ]:
band_to_power_correlation[band]["outlier_removed_df"].shape

- Plotting all of the conditions

In [ ]:
for band in ALL_BANDS:
    for region_1, region_2 in band_to_power_correlation[band]["brain_region_pairs"]:
        region_1_basename = region_1.split("_")[0]
        region_2_basename = region_2.split("_")[0]
        x = band_to_power_correlation[band]["outlier_removed_df"][region_1]
        y = band_to_power_correlation[band]["outlier_removed_df"][region_2]
        
        # Perform linear regression to get the slope, intercept and r-value (correlation coefficient)
        slope, intercept, r_value, p_value, std_err = linregress(x, y)
        
        # Create a line of best fit using the slope and intercept
        line = slope * x + intercept
        
        # Create scatter plot
        sns.scatterplot(x=x, y=y, data=band_to_power_correlation[band]["outlier_removed_df"], hue='outcome_and_trial_or_baseline', palette=BASELINE_OUTCOME_TO_COLOR)
        
        # Plot line of best fit
        plt.plot(x, line, color='red')
        
        # Add R² value to the plot
        plt.text(0.1, 0.9, f'R = {r_value:.2f}', transform=plt.gca().transAxes)
        
        # Add labels and legend
        plt.title("Power correlation of Z-scored {} band LFP: {} and {}".format(band, region_2_basename, region_1_basename))
        plt.xlabel('{} {} power of Z-scored LFP'.format(band, region_1_basename))
        plt.ylabel('{} {} power of Z-scored LFP'.format(band, region_2_basename))
        plt.legend(loc="lower right")
        plt.tight_layout()
        plt.savefig("./proc/power_correlation/zscored/{}/all_condition_{}_{}_power_correlation_of_zscored_{}_lfp.png".format(band, region_1_basename, region_2_basename, band))
        # Display the plot
        plt.show()




In [ ]:
raise ValueError()

In [ ]:
channel_map_and_all_trials_df = filtered_df

In [ ]:
channel_map_and_all_trials_df["trial_outcome"].unique()

In [ ]:
channel_map_and_all_trials_df["trial_or_baseline"]

In [ ]:
for band in ALL_BANDS:
    band_df = band_to_power_correlation[band]["outlier_removed_df"]
    band_to_power_correlation[band]["region_pair_to_outcome_to_r2"] = defaultdict(nested_dict)
    for outcome in band_df["trial_outcome"].unique():
        outcome_df = band_df[band_df["trial_outcome"] == outcome]
        for region_1, region_2 in brain_region_pairs:
            region_1_basename = region_1.split("_")[0]
            region_2_basename = region_2.split("_")[0]
            
            x = outcome_df[region_1]
            y = outcome_df[region_2]
            
            # Perform linear regression to get the slope, intercept and r-value (correlation coefficient)
            slope, intercept, r_value, p_value, std_err = linregress(x, y)
            # Square the r value to get the r squared value
            r2_value = r_value**2
            band_to_power_correlation[band]["region_pair_to_outcome_to_r2"]["{}_{}".format(region_1.split("_")[0], region_2.split("_")[0])][outcome]["r"] = r_value
            band_to_power_correlation[band]["region_pair_to_outcome_to_r2"]["{}_{}".format(region_1.split("_")[0], region_2.split("_")[0])][outcome]["std"] = std_err
            
            # Create a line of best fit using the slope and intercept
            line = slope * x + intercept
            
            # Create scatter plot
            sns.scatterplot(x=x, y=y, data=outcome_df, hue='outcome_and_trial_or_baseline', palette=BASELINE_OUTCOME_TO_COLOR, style='outcome_and_trial_or_baseline', markers=['^', 'o'])
            
            # Plot line of best fit
            plt.plot(x, line, color='red')
            
            # Add R² value to the plot
            plt.text(0.1, 0.9, f'R = {r_value:.2f}', transform=plt.gca().transAxes)
            
            # Add labels and legend
            plt.title("Power Correlation of Z-scored {} LFP: {} and {}".format(band, region_2_basename, region_1_basename))
            plt.xlabel('{} {} Power of Z-scored LFP'.format(region_1_basename, band))
            plt.ylabel('{} {} Power of Z-scored LFP'.format(region_2_basename, band))
            plt.legend(loc="lower right")
            plt.tight_layout()
            plt.savefig("./proc/power_correlation/zscored/{}/{}_{}_{}_power_correlation_of_zscored_{}_lfp.png".format(band, outcome, region_1_basename, region_2_basename, band))
            # Display the plot
            plt.show()

In [ ]:
for band in ALL_BANDS:
    # Convert the nested dictionary to a DataFrame
    data = []
    for group_name, group_data in band_to_power_correlation[band]['region_pair_to_outcome_to_r2'].items():
        for bar_name, bar_dict in group_data.items():
            data.append({"Group": group_name, "Bar": bar_name, "r": bar_dict["r"], "std": bar_dict["std"]})
    df = pd.DataFrame(data)
    
    # Create the bar plot using seaborn
    # sns.catplot(
    #     data=df, 
    #     x='Group', 
    #     y='r2', 
    #     hue='Bar', 
    #     kind='bar', 
    #     height=4, 
    #     aspect=2,
    #     legend=False,
    #     # yerr=df['std'].values,  # This line adds the SEM bars
    #     # capsize=0.1  # This line adds caps on the error bars
    # )
    
    # Create barplot
    ax = sns.barplot(x='Group', y='r', hue='Bar', data=df, ci=None)
    
    # Adding error bars
    groups = df['Group'].unique()
    bars_per_group = df['Bar'].nunique()
    bar_width = 0.8 / bars_per_group
    x_positions = []
    
    for i, group in enumerate(groups):
        num_bars = df[df['Group'] == group].shape[0]
        group_positions = np.linspace(i - bar_width*(num_bars-1)/2, i + bar_width*(num_bars-1)/2, num_bars)
        x_positions.extend(group_positions)
    
    for i, (r2, sem) in enumerate(zip(df['r'], df['std'])):
        plt.errorbar(x_positions[i], r2, yerr=sem, fmt='none', color='black', capsize=5)
    
    
    plt.xticks(rotation=90)
    plt.xlabel("Brain region pairs")
    plt.ylabel("Power correlation r")
    plt.legend(title="Trial Conditions")
    plt.title("{} Power correlations".format(band))
    plt.tight_layout()
    plt.grid()
    
    plt.savefig("./proc/power_correlation/zscored/all_zscored_{}_lfp_power_correlation.png".format(band))
    # Show the plot
    plt.show()

In [ ]:

# Convert the nested dictionary to a DataFrame
data = []
for group_name, group_data in region_pair_to_outcome_to_r2.items():
    for bar_name, bar_dict in group_data.items():
        data.append({"Group": group_name, "Bar": bar_name, "r": bar_dict["r"], "std": bar_dict["std"]})
df = pd.DataFrame(data)

# Create the bar plot using seaborn
# sns.catplot(
#     data=df, 
#     x='Group', 
#     y='r2', 
#     hue='Bar', 
#     kind='bar', 
#     height=4, 
#     aspect=2,
#     legend=False,
#     # yerr=df['std'].values,  # This line adds the SEM bars
#     # capsize=0.1  # This line adds caps on the error bars
# )

# Create barplot
ax = sns.barplot(x='Group', y='r', hue='Bar', data=df, ci=None)

# Adding error bars
groups = df['Group'].unique()
bars_per_group = df['Bar'].nunique()
bar_width = 0.8 / bars_per_group
x_positions = []

for i, group in enumerate(groups):
    num_bars = df[df['Group'] == group].shape[0]
    group_positions = np.linspace(i - bar_width*(num_bars-1)/2, i + bar_width*(num_bars-1)/2, num_bars)
    x_positions.extend(group_positions)

for i, (r2, sem) in enumerate(zip(df['r'], df['std'])):
    plt.errorbar(x_positions[i], r2, yerr=sem, fmt='none', color='black', capsize=5)


plt.xticks(rotation=90)
plt.xlabel("Brain region pairs")
plt.ylabel("Power correlation r")
plt.legend(title="Trial Conditions")
plt.title("Power correlations")
plt.tight_layout()
plt.grid()

plt.savefig("./proc/power_correlation/zscored/all_zscored_lfp_power_correlation.png")
# Show the plot
plt.show()